1) Данные: как собрать 150–200 треков так, чтобы они реально помогли
1.2 Что именно сохранять по каждой песне (минимум)
id, title, album, year, url, lyrics_raw, section_headers_raw
Плюс полезно:
    •	признак версии (live/acoustic/демо)
    •	фиты (если в тексте отмечены)
    •	количество строк / куплетов
1.3 Чистка (очень влияет на качество)
    •	привести секции к единому набору (VERSE, CHORUS, BRIDGE, OUTRO…)
    •	убрать повторяющиеся мусорные блоки и “лишние” подсказки Genius
    •	выкинуть суперкороткие интерлюдии (или пометить отдельным типом)
Цель: чтобы модель училась не на шуме и не на “Embed”.
2) Разметка: делаем стиль контролируемым, а не “магическим”
Для «Макулатуры» важно, чтобы модель держала:
    •	структуру (как устроена песня)
    •	дикцию/образность (длинные строки, разреженная рифмовка, интонация, социально-бытийный тон — если упрощать)
2.1 Каноническая структура в тексте
Вставляй теги:
<VERSE1>…</VERSE1> <CHORUS>…</CHORUS> ...
2.2 Стилевые теги (в промпт, как “ручки управления”)
Считаются автоматом и сохраняются в JSON:
    •	avg_line_len, lines_per_section, repetition_rate
    •	rhyme_density (грубо)
    •	lexical_darkness (словари/частоты маркеров)
    •	slang_level, obscenity_flag (как тег, не цензура)
3) Instruction-датасет: чтобы бот делал именно “новые песни”
С твоим объёмом лучше мешать 2 типа задач.
Тип A — “написать с нуля по ТЗ”

Запрос: тема/сцена/голос (1-е лицо)/структура/длина/настроение/запреты.
Ответ: размеченный текст песни.
4) База (3B RU-friendly) + стек под M1
Самый гладкий путь на M1 обычно один из двух:
Вариант 1 — MLX (самый “родной” для Apple Silicon)
    •	инференс и fine-tune через mlx-lm
    •	удобно держать LoRA и быстро итератить
Вариант 2 — llama.cpp (самый “всеядный” по GGUF)
    •	быстрый инференс в GGUF
    •	LoRA-адаптеры тоже можно подключать в экосистеме llama.cpp/конвертеров
Выбор простой:
    •	если хочешь максимально комфортное обучение на M1 → MLX
    •	если хочешь максимально удобный прод-инференс/кванты → llama.cpp
5) LoRA-тренировка: настройки, которые обычно “встают” с первого раза
Стартовые параметры (как отправная точка):
    •	context: 1024–2048
    •	LoRA rank: 16
    •	lr: 1e-4…2e-4
    •	epochs: 2 (и early stop по val loss/метрикам структуры)

Ключевой момент: тренируй на instruction-формате, а не на “просто продолжи текст”, иначе бот будет хуже слушаться ТЗ и чаще копировать обороты.
6) Генератор: декодинг + проверка структуры
Декодинг:
    •	temperature 0.9–1.1
    •	top_p 0.9–0.95
    •	repetition penalty 1.08–1.15
    •	блокировка повтора n-грамм (если есть)

Пост-процессинг:
    •	есть ли все секции
    •	не пустые ли секции
    •	длина строк/кол-во строк в секциях в заданных рамках
7) Анти-плагиат: делай как “гейт”, а не “опцию”
Три уровня (вместе дают нормальную защиту):
    1.	Шинглы 5–8 слов по всем оригинальным песням → если совпадений много, регенерация
    2.	Fuzzy по строкам (Levenshtein/rapidfuzz) → ловит слегка изменённые цитаты
    3.	Семантика (эмбеддинги строф) → ловит “очень близко по смыслу и форме”

Правило-минимум:
    •	2 подряд строки “похожи сильно” → регенерация сразу
    •	max similarity по корпусу выше порога → регенерация с запретом на найденные строки
8) MVP-план на практике (чтобы быстро увидеть результат)
    1.	Парсер Genius → songs.jsonl
    2.	Чистка + секции → songs_marked.jsonl
    3.	Сбор instruction датасета (A+B) → train/val.jsonl
    4.	Baseline генерация (без LoRA) → замер “похожести”
    5.	LoRA v1 → сравнить с baseline
    6.	Anti-plagiarism v1 (шинглы+fuzzy) → вшить в генератор
    7.	Улучшение датасета (добавить B-задачи) → LoRA v2
9) Структура репозитория (чтобы не утонуть)
    •	data/raw/ (html/json от Genius)
    •	data/clean/ (songs.jsonl)
    •	data/marked/ (songs_marked.jsonl)
    •	data/instruct/ (train.jsonl, val.jsonl)
    •	train/ (скрипты LoRA)
    •	infer/ (генератор + параметры)
    •	filters/ (анти-плагиат индекс + правила)
    •	eval/ (метрики структуры/новизны)
